In [1]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt")

from gensim.models.word2vec import Word2Vec
from gensim.parsing.preprocessing import (
    strip_punctuation,
    strip_numeric,
    strip_short,
    stem_text,
    strip_multiple_whitespaces,
    remove_stopwords,
    STOPWORDS,
)

from bs4 import BeautifulSoup
from collections import defaultdict
#import networkx as nx
import matplotlib.pyplot as plt
#from pyvis.network import Network
import time

# ejecutar dataframe de forma paralela
from pandarallel import pandarallel  # import pandarallel

pandarallel.initialize()  # initialize pandarallel

[nltk_data] Downloading package wordnet to /home/ymamani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ymamani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ymamani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
path_mcdonals_file = "mcdonald_reviews_5k.csv"
ds_mcdonals_reviews = pd.read_csv(path_mcdonals_file, encoding="latin-1")
ds_mcdonals_reviews

,review,rating
0,Why does it look like someone spit on my food?...,1 star
1,It'd McDonalds. It is what it is as far as the...,4 stars
2,Made a mobile order got to the speaker and che...,1 star
3,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,5 stars
4,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...
4995,Excellent,5 stars
4996,Good,4 stars
4997,Neutral,3 stars
4998,Neutral,3 stars


In [3]:
#ds_mcdonals_reviews.to_csv('mcdonald_reviews_5k.csv', index=False)

In [4]:
# ds_mcdonals_reviews = ds_mcdonals_reviews.drop(
#     [
#         "reviewer_id",
#         "store_name",
#         "category",
#         "store_address",
#         #'latitude',
#         "longitude",
#         "rating_count",
#         "review_time",
#     ],
#     axis=1,
# )

In [5]:
#ds_mcdonals_reviews = ds_mcdonals_reviews[["review", "rating"]]

In [3]:
#reemplazando los valores de rating con numeros
replacements = {"1 star": 1, "2 stars": 2, "3 stars": 3, "4 stars":4, "5 stars":5}
ds_mcdonals_reviews.replace(replacements, inplace=True)

/tmp/ipykernel_1558828/794213423.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds_mcdonals_reviews.replace(replacements, inplace=True)


In [4]:
# extender contracciones
contractions_dict = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "this's": "this is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
}

In [5]:
# Función para extender contracciones
def expand_contractions(text):
    text = str(text).lower()
    for contraction, expansion in contractions_dict.items():
        text = text.replace(contraction, expansion)
    return text


# aplicando funcion expand_contractions
ds_mcdonals_reviews["contraction_review"] = ds_mcdonals_reviews[
    "review"
].parallel_apply(expand_contractions)

In [6]:
# funcion para limpieza de textos

def clean_text(text):
    text = str(text).lower()
    #text = BeautifulSoup(text, "lxml").getText()  # remove etiquetas html
    text = re.sub(
        r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE
    )  # Elimina URLs    
    text = re.sub(
        r"\@\w+|\#\w+", "", text
    )  # Elimina las menciones @ y '#' de las redes sociales
    text = re.sub(r" +", " ", text)  # elimina espacios en blanco
    text = strip_punctuation(text)  # Elimina los caracteres de puntuación
    text = strip_numeric(text)  # Elimina los números
    text = re.sub("[^a-zA-Z0-9\s]+","",text)
    text = strip_multiple_whitespaces(text)
    # text = strip_short(text,minsize=2)# Elimina las palabras cortas
    text = re.sub(
        r"\b\w{20,}\b", "", text
    )  # elimina palabras largar de 20 caracteres a mas.
    # text = set(stopwords.words('english'))
    # text = remove_stopwords(text)
    return text


ds_mcdonals_reviews["clean_review"] = ds_mcdonals_reviews[
    "contraction_review"
].parallel_apply(clean_text)

In [7]:
def remove_stopwords_text(text):
    text = text.split()
    sw_text = stopwords.words("english")
    sw_text = [word for word in text if not word in sw_text]
    final_sw_text = " ".join(sw_text)
    return final_sw_text


ds_mcdonals_reviews["sw_review"] = ds_mcdonals_reviews["clean_review"].parallel_apply(
    remove_stopwords_text
)

In [8]:
ds_mcdonals_reviews = ds_mcdonals_reviews.drop(
    columns=["review", "contraction_review", "clean_review"]
)
ds_mcdonals_reviews["polarity"] = ds_mcdonals_reviews["rating"].parallel_apply(
    lambda x: 1 if x >= 4 else 0
)

In [9]:
ds_mcdonals_reviews["polarity"].value_counts()

polarity
1    2693
0    2307
Name: count, dtype: int64

In [10]:
ds_mcdonals_reviews

,rating,sw_review,polarity
0,1,look like someone spit food normal transaction...,0
1,4,would mcdonalds far food atmosphere go staff m...,1
2,1,made mobile order got speaker checked line mov...,0
3,5,mc crispy chicken sandwich customer service qu...,1
4,1,repeat order times drive thru still manage mes...,0
...,...,...,...
4995,5,excellent,1
4996,4,good,1
4997,3,neutral,0
4998,3,neutral,0


In [11]:
import gensim
w2v_embeddings = gensim.models.KeyedVectors.load_word2vec_format('embedding_imdb2k_w2v.txt', binary=False)
glove_embeddings = gensim.models.KeyedVectors.load_word2vec_format('GloVe_vectors_imdb.txt', binary=False)
ft_embeddings = gensim.models.KeyedVectors.load_word2vec_format('embedding_imdb2k_ft.txt', binary=False)
bert_embeddings = gensim.models.KeyedVectors.load_word2vec_format('embedding_imdb2k_bert.txt', binary=False)



In [12]:
# Tokenización y representación de documentos
def document_vector(doc, model):
    words = doc.split()
    vectors = [
        model[word] for word in words if word in model
    ]  # vectors = [embeddings.get(word, np.zeros(300)) for word in words]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # return np.zeros(model.vector_size)  # Vector de ceros si no se encuentran palabras
        return np.zeros(300)

In [14]:
# Aplicar tokenización y representación a todas las reseñas
ds_mcdonals_reviews["vector_sentence_w2v"] = ds_mcdonals_reviews["sw_review"].parallel_apply(lambda x: document_vector(x, w2v_embeddings))
ds_mcdonals_reviews["vector_sentence_glove"] = ds_mcdonals_reviews["sw_review"].parallel_apply(lambda x: document_vector(x, glove_embeddings))
ds_mcdonals_reviews["vector_sentence_ft"] = ds_mcdonals_reviews["sw_review"].parallel_apply(lambda x: document_vector(x, ft_embeddings))
ds_mcdonals_reviews["vector_sentence_bert"] = ds_mcdonals_reviews["sw_review"].parallel_apply(lambda x: document_vector(x, bert_embeddings))
#34min


In [15]:
x_ds_mcdonals_reviews_w2v = np.vstack(ds_mcdonals_reviews["vector_sentence_w2v"])
y_ds_mcdonals_reviews_w2v = ds_mcdonals_reviews["polarity"]

x_ds_mcdonals_reviews_glove = np.vstack(ds_mcdonals_reviews["vector_sentence_glove"])
y_ds_mcdonals_reviews_glove = ds_mcdonals_reviews["polarity"]

x_ds_mcdonals_reviews_ft = np.vstack(ds_mcdonals_reviews["vector_sentence_ft"])
y_ds_mcdonals_reviews_ft = ds_mcdonals_reviews["polarity"]

x_ds_mcdonals_reviews_bert = np.vstack(ds_mcdonals_reviews["vector_sentence_bert"])
y_ds_mcdonals_reviews_bert = ds_mcdonals_reviews["polarity"]


In [16]:
from sklearn.model_selection import train_test_split, cross_validate

x_train_w2v, x_test_w2v, y_train_w2v, y_test_w2v = train_test_split(
    x_ds_mcdonals_reviews_w2v,
    y_ds_mcdonals_reviews_w2v,
    test_size=0.2,
    random_state=42,
)

x_train_glove, x_test_glove, y_train_glove, y_test_glove = train_test_split(
    x_ds_mcdonals_reviews_glove,
    y_ds_mcdonals_reviews_glove,
    test_size=0.2,
    random_state=42,
)

x_train_ft, x_test_ft, y_train_ft, y_test_ft = train_test_split(
    x_ds_mcdonals_reviews_ft,
    y_ds_mcdonals_reviews_ft,
    test_size=0.2,
    random_state=42,
)

x_train_bert, x_test_bert, y_train_bert, y_test_bert = train_test_split(
    x_ds_mcdonals_reviews_bert,
    y_ds_mcdonals_reviews_bert,
    test_size=0.2,
    random_state=42,
)


In [17]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import (
    accuracy_score,
    accuracy_score,
    roc_auc_score,
    f1_score,
    classification_report,
    precision_score,
    recall_score,
)

In [18]:
def Evaluation(model, X_train, X_test, y_train, y_test, hypertuning=False):
    
    y_pred = model.predict(X_train)
    y_pred_proba = model.predict_proba(X_train)

    accuracy_train = accuracy_score(y_train, y_pred)
    precision_train = precision_score(y_train, y_pred)
    recall_train = recall_score(y_train, y_pred)
    F1_score_train = f1_score(y_train, y_pred)
    # print("F1_Score = ", F1_score_train )
    roc_auc_train = roc_auc_score(y_train, y_pred_proba[:, 1])
    # print( classification_report( y_train, y_pred ) )

    # print( " For Test Set :  ")
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy_test = accuracy_score(y_test, y_pred)
    precision_test = precision_score(y_test, y_pred)
    recall_test = recall_score(y_test, y_pred)
    F1_score_test = f1_score(y_test, y_pred)
    # print("F1_Score = ", F1_score_test )
    roc_auc_test = roc_auc_score(y_test, y_pred_proba[:, 1])
    
    #cross-validation
    cross_val = cross_val_score(model, X_train, y_train, cv=5).mean()
    
    return (
        accuracy_train,
        precision_train,
        recall_train,
        F1_score_train,
        roc_auc_train,
        accuracy_test,
        precision_test,
        recall_test,
        F1_score_test,
        roc_auc_test,
        cross_val
    )

In [19]:
def apply_models_with_default_paramters(X_train, X_test, y_train, y_test):
    models_default = [
        {"ModelNames": "SVM", "Model": SVC(kernel="poly", probability=True)},
        {
            "ModelNames": "RF",
            "Model": RandomForestClassifier(n_estimators=100, random_state=42),
        },
        {"ModelNames": "KNN", "Model": KNeighborsClassifier(n_neighbors=5)},
        {"ModelNames": "XGB", "Model": XGBClassifier()},
        {"ModelNames": "LR", "Model": LogisticRegression(max_iter=1000)},
    ]
    
    cross_val_train = []

    F1_Score_train = []
    Accuracy_train = []
    Recall_train = []
    Precision_train = []
    ROC_AUC_Score_train = []

    F1_Score_test = []
    Accuracy_test = []
    Recall_test = []
    Precision_test = []
    ROC_AUC_Score_test = []

    Model_Name = []

    for model in models_default:
        # print(model)
        Model_Name.append(model["ModelNames"])
        model["Model"].fit(X_train, y_train)

        (
            accuracy_train,
            precision_train,
            recall_train,
            F1_score_train,
            roc_auc_train,
            accuracy_test,
            precision_test,
            recall_test,
            F1_score_test,
            roc_auc_test,
            cross_val            
        ) = Evaluation(model["Model"], X_train, X_test, y_train, y_test, False)

        cross_val_train.append(cross_val)
        
        F1_Score_train.append(F1_score_train)
        Accuracy_train.append(accuracy_train)
        Recall_train.append(recall_train)
        Precision_train.append(precision_train)
        ROC_AUC_Score_train.append(roc_auc_train)

        F1_Score_test.append(F1_score_test)
        Accuracy_test.append(accuracy_test)
        Recall_test.append(recall_test)
        Precision_test.append(precision_test)
        ROC_AUC_Score_test.append(roc_auc_test)

    results = pd.DataFrame()
    results["Model_Name"] = Model_Name

    train_test_f1_score_difference = np.subtract(
        F1_Score_train, F1_Score_test
    )  # To Check Overfitting/Underfitting

    results["Cross validation mean"] = cross_val_train
    
    results["Accuracy on Test Set"] = Accuracy_test
    results["Precision on Test Set"] = Precision_test
    results["Recall on Test Set"] = Recall_test
    results["F1_Score on Test Set"] = F1_Score_test         
    results["ROC_AUC_Score on Test Set"] = ROC_AUC_Score_test

    results["Accuracy on Train Set"] = Accuracy_train
    results["Precision on Train Set"] = Precision_train
    results["Recall on Train Set"] = Recall_train
    results["F1_Score on Train Set"] = F1_Score_train      
    results["ROC_AUC_Score on Train Set"] = ROC_AUC_Score_train
        
    results["Difference of F1_Score on train and test"] = train_test_f1_score_difference

    results = results.sort_values(
        by=["F1_Score on Test Set", "Difference of F1_Score on train and test"],
        ascending=[False, True],
    )

    return results

In [23]:
def apply_model_plot(df_model):
    # Generar gráfico

    plt.figure(figsize=(8, 5))
    # plt.bar(models, accuracies, color=['orange', 'green', 'red'])
    bars = plt.bar(df_model['Model_Name'], df_model['Accuracy on Test Set'], color=["orange", "green", "red", "blue"])
    plt.ylim(0, 1.0)
    plt.xlabel("Modelo")
    plt.ylabel("Accuracy")
    plt.title("Accuracy test de Modelos de Sentimiento")
    for bar, accuracy in zip(bars, df_model['Accuracy on Test Set']):
        plt.text(
            bar.get_x() + bar.get_width() / 2 - 0.08,
            bar.get_height() + 0.02,
            f"{accuracy:.4f}",
            fontsize=12,
        )

    plt.show()

In [20]:
#x_train_w2v, x_test_w2v, y_train_w2v, y_test_w2v
Results_w2v = apply_models_with_default_paramters(x_train_w2v, x_test_w2v, y_train_w2v, y_test_w2v)
Results_w2v

,Model_Name,Cross validation mean,Accuracy on Test Set,Precision on Test Set,Recall on Test Set,F1_Score on Test Set,ROC_AUC_Score on Test Set,Accuracy on Train Set,Precision on Train Set,Recall on Train Set,F1_Score on Train Set,ROC_AUC_Score on Train Set,Difference of F1_Score on train and test
1,RF,0.79150,0.804,0.859833,0.761111,0.807466,0.891359,0.97425,0.973223,0.979099,0.976152,0.992196,0.168686
3,XGB,0.78975,0.791,0.837067,0.761111,0.797284,0.880680,0.97450,0.974988,0.977706,0.976345,0.996984,0.179061
2,KNN,0.76425,0.772,0.809524,0.755556,0.781609,0.838424,0.83225,0.858664,0.823967,0.840958,0.915944,0.059348
4,LR,0.76575,0.779,0.884337,0.679630,0.768586,0.853720,0.76975,0.869305,0.673479,0.758964,0.851030,-0.009623
0,SVM,0.75275,0.763,0.924370,0.611111,0.735786,0.857512,0.75750,0.922198,0.600093,0.727068,0.853206,-0.008718


In [21]:
Results_glove = apply_models_with_default_paramters(x_train_glove, x_test_glove, y_train_glove, y_test_glove)
Results_glove

,Model_Name,Cross validation mean,Accuracy on Test Set,Precision on Test Set,Recall on Test Set,F1_Score on Test Set,ROC_AUC_Score on Test Set,Accuracy on Train Set,Precision on Train Set,Recall on Train Set,F1_Score on Train Set,ROC_AUC_Score on Train Set,Difference of F1_Score on train and test
1,RF,0.80100,0.824,0.874486,0.787037,0.828460,0.883355,0.96275,0.971308,0.959127,0.965179,0.983835,0.136719
3,XGB,0.79850,0.812,0.852000,0.788889,0.819231,0.877216,0.96325,0.971335,0.960056,0.965662,0.993825,0.146431
2,KNN,0.78050,0.790,0.846639,0.746296,0.793307,0.842506,0.83800,0.877572,0.812355,0.843705,0.914622,0.050398
4,LR,0.78650,0.790,0.876712,0.711111,0.785276,0.863690,0.79450,0.877910,0.718068,0.789985,0.879034,0.004709
0,SVM,0.71075,0.698,0.919014,0.483333,0.633495,0.842120,0.72850,0.954043,0.520669,0.673678,0.881501,0.040183


In [22]:
Results_ft = apply_models_with_default_paramters(x_train_ft, x_test_ft, y_train_ft, y_test_ft)
Results_ft

,Model_Name,Cross validation mean,Accuracy on Test Set,Precision on Test Set,Recall on Test Set,F1_Score on Test Set,ROC_AUC_Score on Test Set,Accuracy on Train Set,Precision on Train Set,Recall on Train Set,F1_Score on Train Set,ROC_AUC_Score on Train Set,Difference of F1_Score on train and test
3,XGB,0.80150,0.815,0.852883,0.794444,0.822627,0.870010,0.96325,0.971778,0.959591,0.965646,0.993867,0.143019
1,RF,0.80000,0.804,0.865957,0.753704,0.805941,0.883311,0.96275,0.971308,0.959127,0.965179,0.983857,0.159238
4,LR,0.79000,0.778,0.842672,0.724074,0.778884,0.859000,0.79250,0.853177,0.742220,0.793840,0.865370,0.014956
2,KNN,0.78200,0.772,0.820988,0.738889,0.777778,0.828979,0.84250,0.884402,0.813748,0.847605,0.914080,0.069827
0,SVM,0.78975,0.779,0.873536,0.690741,0.771458,0.858820,0.79500,0.878049,0.718997,0.790603,0.865288,0.019145


In [23]:
Results_bert = apply_models_with_default_paramters(x_train_bert, x_test_bert, y_train_bert, y_test_bert)
Results_bert

,Model_Name,Cross validation mean,Accuracy on Test Set,Precision on Test Set,Recall on Test Set,F1_Score on Test Set,ROC_AUC_Score on Test Set,Accuracy on Train Set,Precision on Train Set,Recall on Train Set,F1_Score on Train Set,ROC_AUC_Score on Train Set,Difference of F1_Score on train and test
4,LR,0.8330,0.838,0.890496,0.798148,0.841797,0.915451,0.85750,0.895948,0.831863,0.862717,0.931547,0.020920
3,XGB,0.8215,0.828,0.875510,0.794444,0.833010,0.913027,0.97475,0.975440,0.977706,0.976572,0.997788,0.143562
1,RF,0.8190,0.819,0.881104,0.768519,0.820969,0.907031,0.97425,0.974098,0.978170,0.976130,0.992243,0.155160
0,SVM,0.8145,0.818,0.897778,0.748148,0.816162,0.893458,0.83275,0.904580,0.770553,0.832205,0.912460,0.016043
2,KNN,0.7925,0.802,0.859244,0.757407,0.805118,0.871137,0.84875,0.886228,0.824895,0.854462,0.931328,0.049344
